Programming Homework 2 Instructions 

In a practical, we saw Python code implementing the Boyer-Moore algorithm. Some of the code is for preprocessing the pattern P into the tables needed to execute the bad character and good suffix rules — we did not discuss that code. But we did discuss the code that performs the algorithm given those tables:**bold text**

In [ ]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences


Measuring Boyer-Moore's benefit. First, download the Python module for Boyer-Moore preprocessing:

http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py

This module provides the BoyerMoore class, which encapsulates the preprocessing info used by the boyer_moore function above. Second, download the provided excerpt of human chromosome 1:

http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

Third, implement versions of the naive exact matching and Boyer-Moore algorithms that additionally count and return (a) the number of character comparisons performed and (b) the number of alignments tried. Roughly speaking, these measure how much work the two different algorithms are doing.

For a few examples to help you test if your enhanced versions of the naive exact matching and Boyer-Moore algorithms are working properly, see these notebooks:

* Naive

* Boyer-Moore

Programming Homework 2

Q1: How many alignments does the naive exact matching algorithm try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [1]:
#Read Genome from FASTA file#
#############################

def readGenome(filename):
    """ This function reads a FASTA file"""
    genome = '' # initialize genome to empty string
    with open(filename,'r') as f:               # Open file as f
        for line in f:                          # Loop therough and read each line of file f
            if not line[0] == '>':              # If line does not start with ">"
                genome += line.rstrip()         # Add line to the string genome, rstrip removes trailing whitespace from ends of string        
        
    return genome                               # After reading and adding all lines return the string genome



In [5]:
#Import google drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
%cd /content/gdrive/My\ Drive/Genomic\ Data\ Science/Algorithms_for_DNA_Sequencing/Week2

/content/gdrive/My Drive/Genomic Data Science/Algorithms_for_DNA_Sequencing/Week2


In [7]:
genome = readGenome('chr1.GRCh38.excerpt.fasta')
genome[:10]

'TTGAATGCTG'

In [8]:
t = genome
p = 'GGCGCGGTGGCTCACGCCTGTAAT'

In [9]:
import bisect

Substring Index

In [10]:
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [11]:
def query_index(p, t, index):
    k = index.k
    offsets =[]
    for i in index.query(p):
        if p[k:] == t[i+k:i+len(p)]:
            offsets.append(i)
    return offsets           

In [12]:
index = Index(t, 8)

In [13]:
# Pigeon hole using Substring Index
def approximate_match(p, t, n):   #Takes as arguments pattern, text and the max number of mismatches n
    segment_length = int(round(len(p)/(n+1))) # Set the length of each partition of p, convert to int so that indices are int or it will raise error
    all_matches = set() # create a set to hold all the indices wehere we find a match, wihtout duplicates
    for i in range(n+1): # For each segment of P, for each iteration move along by the lenght of 1 segment
        #Set bounds of P for the segment we are searching for 
        start = i * segment_length # so if i = 0 and seg_l = 2, start = 0, i = 1, seg_l= 2, start = 1 * 2
        end = min((i+1) * segment_length, len(p)) #to make sure we dont run over end of p, since p might not be a perfect multiple of n+1
        index = Index(t, 8)
        matches = query_index(p[0:len(p-1)], t, index)
        
        # Verification to see that the rest of p matches t with no more than n mismatches
        for m in matches:
            #Make sure our location does not let p run off the begining or the end of t
            if m < start or m-start+len(p) > len(t): # if any of this is true then we will run past the beginning or end of t
                continue # if any of the abov is true skip the rest of the loop
                
            mismatch = 0 # To count the mismatches between the rest of p and t
            # Compare part of p before the start(from 0 to start against corresponding position in t)
            for j in range(0, start):
                if not p[j] == t[m-start + j]: # if corresponding positions dont match
                    mismatch += 1 # increment mismatch by 1
                    if mismatch > n: # If the number of mismatches is more than n
                        break  # break out of this inner loop
            # Compare the part of p after the end
            for j in range(end, len(p)): 
                if not p[j] == t[m-start+j]:
                    mismatch += 1
                    if mismatch > n:
                        break
                        
            if mismatch <= n: # If we have verified on both sides of p and mismatches are less than n
                all_matches.add(m-start) # we add the m - start to get the begining of p for the match to the set all_matches
                
                
                    
    return list(all_matches)         
    

In [16]:
# Naive exact match

def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences


In [17]:
# Pigeonhole using naive
def approximate_match(p, t, n):   #Takes as arguments pattern, text and the max number of mismatches n
    segment_length = int(round(len(p)/(n+1))) # Set the length of each partition of p, convert to int so that indices are int or it will raise error
    all_matches = set() # create a set to hold all the indices wehere we find a match, wihtout duplicates
    hits = []
    for i in range(n+1): # For each segment of P, for each iteration move along by the lenght of 1 segment
        #Set bounds of P for the segment we are searching for 
        start = i * segment_length # so if i = 0 and seg_l = 2, start = 0, i = 1, seg_l= 2, start = 1 * 2
        end = min((i+1) * segment_length, len(p)) #to make sure we dont run over end of p, since p might not be a perfect multiple of n+1
        matches = naive(p[start:end], t)
        # Verification to see that the rest of p matches t with no more than n mismatches
        for m in matches:
            #Make sure our location does not let p run off the begining or the end of t
            if m < start or m-start+len(p) > len(t): # if any of this is true then we will run past the beginning or end of t
                continue # if any of the abov is true skip the rest of the loop
                
            mismatch = 0 # To count the mismatches between the rest of p and t
            # Compare part of p before the start(from 0 to start against corresponding position in t)
            for j in range(0, start):
                if not p[j] == t[m-start + j]: # if corresponding positions dont match
                    mismatch += 1 # increment mismatch by 1
                    if mismatch > n: # If the number of mismatches is more than n
                        break  # break out of this inner loop
            # Compare the part of p after the end
            for j in range(end, len(p)): 
                if not p[j] == t[m-start+j]:
                    mismatch += 1
                    if mismatch > n:
                        break
                        
            if mismatch <= n: # If we have verified on both sides of p and mismatches are less than n
                all_matches.add(m-start) # we add the m - start to get the begining of p for the match to the set all_matches
                
                
                    
    return matches, list(all_matches)         
    

In [18]:
matches, all_matches = approximate_match(p, t, 2)

In [19]:
print(len(matches))

60


In [20]:
# naive_2mm: Naive Matching algorithm with upto 2 mismatches

def naive_2mm(p, t, maxDist):
    occurrences = []
    for i in range(len(t) - len(p) + 1):
        match = True
        nmm = 0
        for j in range(len(p)):
            if t[i+j] != p[j]:
                nmm += 1
                if nmm > maxDist:
                    break
        if nmm <= maxDist:
            occurrences.append(i)
    return occurrences        

In [21]:
naive2mm_matches = naive_2mm(p, t, 2)

In [22]:
len(naive2mm_matches)

19

In [23]:
print(naive2mm_matches)

[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]
